In [14]:
import numpy as np
import pandas as pd
import displacement_solver
import constitutive
import mesh_gen
import quadrature
import stress_gauss
import patch_n_int_nodes
import gauss_pt_coord
import stress_nodes_dc
import spr_stress


In [52]:
# ............................Inputs Parameters......................#

#Doamin geometry
domain_coord = np.array([[2, 0], [20, 0], [20, 10], [0, 10], [0, 2]]);

# Body force components
b = np.array([[0], [0]])

# Traction Components
q = 1/8      #unirt force for unit length

T = np.array([[q, 0]]);

# Young's modulus
E = 1.0

# Poission's ration
nu = 1/3;

# problem type (0--->plane stress, 1---->plane strain)
problem_type = 0;

#Element type used for meshing (0---->4 nodes quadrilateral)
el_type = 0;

# No. of Gauss points required forintegration
ngp2d = 1;
ngp1d = 2;

# Number of mesh in one direction.
N = 320;

In [53]:
u = displacement_solver.solve_fem_plat_with_hole(N, E, nu, ngp2d, ngp1d, el_type, problem_type, domain_coord, b, T)
# print(u_node)

In [54]:
# ms = mesh size ----> Number of element in one direction
ms = N;

#total number of elements
nel = 2*ms*ms 

# reshaping u into u_nodes with displacement in x-direction in first column and y-direciton in second column
u_nodes = u.reshape((2*(ms+1)*(ms+1)-(ms+1), 2))
# print(u_nodes)

In [55]:
# constitutive relation matrix, calculated using the fuction "Constitutube" with input E->(Young's Modulus), nu(Poisson's raton), problem_type(plane stress or plane strain)
C = constitutive.constitutive(E, nu, problem_type)
# print(C);

# mesh_obj = object created to calculate 

In [56]:
# mesh generation 
nx = ms #number of element in x-direction
ny = ms #number of element in y-direction

# el_type = element type specifiedc. 0----> q4, q-----> q8, 2-----> q9
el_type = 0;

#mesh_obj = object created to calculated nodal coordinates ans connectivity array using functon "connectivity" and "coord_array" 
#input nx->number of element in x-direction, xy->number of element in y-direction, domain_coord->coordinates of the corner points and mid-points of the cook's skew beam problem, el_type->element type specidied.

mesh_obj = mesh_gen.platWithHole(nx, ny, domain_coord.reshape(10, 1), el_type);
connect = mesh_obj.connectivity();
# print(connect)
coord = mesh_obj.coord_array();
print(len(coord))
coord_df = pd.DataFrame(coord, columns=['X', 'Y'])
# print(coord_df.head);
coord_df.to_csv('Coord/mesh_'+str(nx)+'.csv')


205761


In [57]:
# from matplotlib import pyplot as plt
# xs = [x[0] for x in coord]
# ys = [x[1] for x in coord]
# plt.scatter(xs, ys)
# plt.show()

In [58]:
#Stress calculation at gauss points
#Stress (3D array) = stress at the gauss points with sigma_xx in the first column, sigma_yy in the seond column ans sigma_xy in the third column. rows number indicates the element number. layer number indicates the gauss point number.

#strains (3-D array) = strains at the gauss points with strain_xx in the first column, strain_yy in the second column and strain_xy in the third column. rows number indicates the gauss point number.

#calculated using the function "get_elemnt_stress" from the class "stress_gauss" with the following inputs;
# i->element number
# npg2d -> number of gauss points in one direction.
# el_type -> element type sepcified.
# connect -> connectivity matrix
# coord -> nodal coordinates array
# u -> displacement at nodal coordinates
# C->constitutive relation matrix;

stress = np.zeros((nel, ngp2d*ngp2d, 3));
strains = np.zeros((nel, ngp2d*ngp2d, 3));

for i in range(nel):
    stress_i_g = np.zeros((ngp2d*ngp2d, 3));
    
    strains_i_g = np.zeros((ngp2d*ngp2d, 3));

    stress_i_g, strains_i_g = stress_gauss.get_element_stress(i, ngp2d, el_type, connect, coord, u, C);
    stress[i][:][:] = stress_i_g;
    strains[i][:][:] = strains_i_g.reshape((1,3));



In [59]:
# Gauss Point coordinates
gauss_coords = np.zeros((nel, ngp2d*ngp2d, 2));


# gp = gauss points in the master domains;
# "gp" and weights" calculated using the function "quadrature" of class "quadrature"

# inputs to the function:
# ngp2d->number of gauss points in one direction.
# gauss points co-ordinates sotes in the variable "gauss_coords" with x-coordiante in the column and y-coordiante in the second column.
# gauss coordinates are calculated using the function "gauss_coords" of class "gauss_pt_coord"ArithmeticError
#inputs ngp2d->number of gauss points in one direction, vertex_coord->nodal coordinates of the current element in for loop, gp->gauss points in the master domain. el_type-> element type specified.

gp = quadrature.quadrature(ngp2d)[0];
for i in range(nel):
    node = connect[i, :];
    vertex_coord = coord[node,:].reshape(-1);
    gauss_coords[i][:][:] = gauss_pt_coord.gauss_pts(ngp2d, vertex_coord, gp, el_type);
# print(len(gauss_coords))
gauss_coords = gauss_coords.reshape(gauss_coords.shape[0], -1);
# print(len(gauss_coords));




In [60]:
#saving stress at gauss points and coordinages of gauss points into csv file for any training or plotting purpose

#reshaping stress (3D) array into 2-D array for the case in which only one gauss point is used
stress_1gp = stress.reshape((stress.shape[0], -1));
# print(stress_1gp);

#creating pandas dataframe and saving the stress data into the folder "Data"
stress_df = pd.DataFrame(stress_1gp, columns = ['sigma_x_ref', 'sigma_y_ref', 'sigma_xy_ref']);
# print(stress_df.head())
stress_df.to_csv('Data/superconv_gauss_stress_ms_'+str(nx)+'.csv');

#creating pandas dataframe and saving the stress coordinate into the folder "Coord"
gauss_poing_coord_df = pd.DataFrame(gauss_coords, columns = ['X', 'Y']);
# print(gauss_coords);
gauss_poing_coord_df.to_csv('Coord/coord_gauss_pt_'+str(nx)+'.csv');


In [61]:
# creation of patches for the implementation of the superconvergent patch recovery(spr_stress) Technique;
# Input:- ms -> mesh size;
# Output:- n_patches -> number of patches in the domain, Patch -> 2-D array with each row representing a patch of elements (4 elements in one patch)., int_nodes->all the internal nodes in the domains
# import importlib
# importlib.reload(patch_n_int_nodes)
patch, n_patches, int_nodes = patch_n_int_nodes.patch_n_int_nodes_plat_with_hole(ms);
# print(int_nodes);
# print(patch);
# print(n_patches);

In [62]:
#Directly calculated stress at the nodes.
# "stress_dc" -> directly calcualted stress at the nodes from FEM.
# "strain_dc" -> directly calculated strain at nodes form FEM.
# calculated using the function "stress_dc" with the following inputs:
# connect-> connectivity matrix, coord->nodal coordinates, u->nodal displacements, nel->total number of elements for the given mesh size, el_type->elements type specified, C = constitutive relation matrix.

stress_dc, strain_dc = stress_nodes_dc.stress_dc(connect, coord, u, nel, el_type, C);
# print(len(strain_dc));

In [63]:
# 2-D array to store the coordinates of the nodes and corresponding stresses
nodes_coord_stress_dc = np.hstack((coord, stress_dc));

# store the array of directly calculated stress and coordinates of nodes in the folder "Overall",  subfolder "DC".
nodes_coord_stress_dc_df = pd. DataFrame(nodes_coord_stress_dc, columns = ['x', 'y', 'sigma_x_dc', 'sigma_y_dc', 'sigma_xy_dc']).round(decimals = 14);
# print(nodes_coord_stress_dc_df);
nodes_coord_stress_dc_df.to_csv('stress/dc/dc_ms_'+str(nx)+'.csv');



In [64]:
#..............stress calculated at the nodes using the spr_stress methd..............#
stress_spr = spr_stress.spr_plat_with_hole(gauss_coords, coord, connect, stress, int_nodes, n_patches ,patch, ms);
# print(len(stress_spr));
# print(len(coord))

#store the calculated spr_stress stress at the nodes for the given nodes for given mesh size "ms" in the folder "overall", and subfolder "spr"
nodes_coord_stress_spr = np.hstack((coord, stress_spr))
# print(nodes_coord_stress_spr)
nodes_coord_stress_spr_df = pd. DataFrame(nodes_coord_stress_spr, columns = ['x', 'y', 'sigma_x_spr', 'sigma_y_spr', 'sigma_xy_spr']).round(decimals = 14);
# print(nodes_coord_stress_spr_df.head());
nodes_coord_stress_spr_df.to_csv('stress/spr/spr_ms_'+str(nx)+'.csv');



In [69]:
print(len(stress_spr))
print(len(nodes_coord_stress_spr))
# ref_stress = nodes_coord_stress_spr[::65,:]
ref_stress = [];
# print(ref_stress)
for i in range(11):
    current_node = 321*64*i
    for j in range(6):
        ref_stress.append(nodes_coord_stress_spr[current_node, :]);
        current_node+=64

ref_stress = np.array(ref_stress)
nodes_coord_stress_spr_ref_df = pd.DataFrame(ref_stress, columns=['x', 'y', 'sigma_x_spr', 'sigma_y_spr', 'sigma_xy_spr']).round(decimals = 14);
nodes_coord_stress_spr_ref_df.to_csv('ref_stress/ref_stress_for_ms_5_from_320.csv')

205761
205761


In [66]:
ape = np.zeros((len(stress_spr), 3));


# print(ape_sigma_x)
for i in range(len(stress_spr)):
    ape[i][0] = abs(stress_spr[i][0]-stress_dc[i][0]);
    ape[i][1] = abs(stress_spr[i][1]-stress_dc[i][1]);
    ape[i][2] = abs(stress_spr[i][2]-stress_dc[i][2]);

# print(ape);

In [67]:
# strs_ref_18 = pd.read_csv('stress/spr/spr_ms_18.csv')
# strs_ref_18 = strs_ref_18.loc[:, ~strs_ref_18.columns.str.contains('Unnamed')];
# strs_ref_18 = strs_ref_18.values
# strs_ref_31 = pd.read_csv('stress/spr/spr_ms_31.csv')
# strs_ref_31 = strs_ref_31.loc[:, ~strs_ref_31.columns.str.contains('Unnamed')];
# strs_ref_31 = strs_ref_31.values
# strs_ref_40 = pd.read_csv('stress/spr/spr_ms_40.csv')
# strs_ref_40 = strs_ref_40.loc[:, ~strs_ref_40.columns.str.contains('Unnamed')];
# strs_ref_40 = strs_ref_40.values
# strs_ref_53 = pd.read_csv('stress/spr/spr_ms_53.csv')
# strs_ref_53 = strs_ref_53.loc[:, ~strs_ref_53.columns.str.contains('Unnamed')];
# strs_ref_53 = strs_ref_53.values
# strs_ref_83 = pd.read_csv('stress/spr/spr_ms_83.csv')
# strs_ref_83 = strs_ref_83.loc[:, ~strs_ref_83.columns.str.contains('Unnamed')];
# strs_ref_83 = strs_ref_83.values
# strs_ref_117 = pd.read_csv('stress/spr/spr_ms_117.csv')
# strs_ref_117 = strs_ref_117.loc[:, ~strs_ref_117.columns.str.contains('Unnamed')];
# strs_ref_117 = strs_ref_117.values
# strs_ref_134 = pd.read_csv('stress/spr/spr_ms_134.csv')
# strs_ref_134 = strs_ref_134.loc[:, ~strs_ref_134.columns.str.contains('Unnamed')];
# strs_ref_134 = strs_ref_134.values
# strs_ref_143 = pd.read_csv('stress/spr/spr_ms_143.csv')
# strs_ref_143 = strs_ref_143.loc[:, ~strs_ref_143.columns.str.contains('Unnamed')];
# strs_ref_143 = strs_ref_143.values
# merged_arr = np.concatenate([strs_ref_18, strs_ref_31, strs_ref_40, strs_ref_53, strs_ref_83, strs_ref_117, strs_ref_134, strs_ref_143])
# print(len(merged_arr))
# nodes_coord_stress_spr_df_18_143 = pd. DataFrame(merged_arr, columns = ['x', 'y', 'sigma_x_spr', 'sigma_y_spr', 'sigma_xy_spr']).round(decimals = 14);
# print(nodes_coord_stress_spr_df_18_143)
# nodes_coord_stress_spr_df_18_143.to_csv('stress/spr/spr_ms_18_to_143.csv');
# ttl_size = len(strs_ref_18)+len(strs_ref_31) + len(strs_ref_40) + len(strs_ref_53)+len(strs_ref_83)+len(strs_ref_117)+len(strs_ref_134)+len(strs_ref_143)
# print(ttl_size)

